In [29]:
#Converting uint16 binary video format into MP4

In [30]:
import imageio
import random
import numpy as np
import warnings
warnings.filterwarnings('ignore') #will get warning about conversion of uint16 to uint8

In [31]:
inputbinaryfile='20180702_1' #Input file name and exclude the .bin extension 
height=480
width=640
fps=90
background_chunk=100 #number of frames that will be used to calculate the background

In [32]:
inputfilename=inputbinaryfile+'.bin'
outputfilename=inputbinaryfile+'.mp4'

In [33]:
rawframes = (np.fromfile(inputfilename, dtype=np.uint16)).reshape(-1,height,width) #A highly efficient way of reading binary data with a known data-type, as well as parsing simply formatted text files. Data written using the tofile method can be read using this function.
number_of_frames,video_height,video_width=rawframes.shape[0],rawframes.shape[1],rawframes.shape[2];
#set number of frames equal to the first element in reshaped.shape
#set video height to the second element in reshaped.shape
#set video width to the third element in reshaped.shape
print('The shape of the video is {0} by {1} and it has {2} frames'.format(video_height,video_width,number_of_frames))
#note that the nubmers below are one greater than the actual indices that exist
#there is no reshaped[number_of_frames,height,width], the last frame is reshaped[number_of_frames-1,height-1,width-1]

The shape of the video is 480 by 640 and it has 2000 frames


In [34]:
#For my background calculation I want to use background_chunk number of frames
#This strategy only works if the object of interest is in motion!
#It's actually much better to first record a background and THEN record your object of interest (which is dont in other scripts)
background_step=int(number_of_frames/background_chunk) #I will generate a list of indices to calculate the background. They will be equidistant
idx=[] #here is the empty list
for i in range(0,number_of_frames,background_step):
    idx.append(i)

In [35]:
#backgroundmean=np.median(rawframes[0:background_chunk], axis=0) #may be better to use median
#I'm using the frames at position idx to calculate the background
backgroundmedian=np.median(rawframes[idx], axis=0) #may be better to use median 
backgroundvariance=np.var(rawframes[idx], axis=0)

In [36]:
backgroundmultiplier=np.abs(rawframes - backgroundmedian) - (9*np.sqrt(backgroundvariance)) #if its 5 standard its NOISE
#we expect positive values for high differences. make them stay by multiplying by 1
backgroundmultiplier[backgroundmultiplier>0]=1
#we expect negative values or no change for small differences. probably just noise, get rid of them by multiplying by 0
backgroundmultiplier[backgroundmultiplier<=0]=0
#apply our expectations to the matrix
rawframes_backgroundsubtracted=np.multiply(rawframes,backgroundmultiplier)

In [37]:
rawframes_backgroundsubtracted=np.multiply(rawframes,backgroundmultiplier)

In [38]:
imageio.mimwrite(outputfilename, rawframes_backgroundsubtracted , fps = fps)

In [39]:
from IPython.display import HTML

In [40]:
#creating a simlink to visualize the movie in jupyter notebook
!ln -s /Users/ranyoha/Real_Sensible_Moseq/data_acquisition/20180702_1.mp4 /Users/ranyoha/Real_Sensible_Moseq/data_acquisition/sim_20180702_1.mp4

In [41]:
HTML("""
<video width="640" height="480" controls>
  <source src="sim_20180702_1.mp4" type="video/mp4">
</video>
""")